In [ ]:
# Seção do código de equação da reta
import numpy as np
from scipy import stats

# Dados de calibração
leituras_ldr = [25.73, 35.80, 66.51]
turbidez = [0, 100, 800]

# Ajuste do modelo de regressão linear
slope, intercept, r_value, p_value, std_err = stats.linregress(leituras_ldr, turbidez)

# Função de regressão linear ajustada
def regressao_linear(x):
    return max(slope * x + intercept, 0)

# Seção do código de leitura e inserção
import serial
import datetime
import mysql.connector
import math

# Configuração da conexão
config = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'sin499-turbidimetro',
    'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

led = 13
ldr = 'A0'
serial_port = serial.Serial('COM3', 9600)

GAMMA = 0.7
RL10 = 50

# Função para ler o valor do sensor LDR e converter para Lux
def read_ldr_and_lux():
    serial_port.write(ldr.encode())
    response = serial_port.readline().decode().strip()
    if response.startswith('Valor LDR: '):
        analogValue = int(response.replace('Valor LDR: ', ''))
        voltage = analogValue / 1024. * 5
        if voltage >= 5:  # Se a tensão for >= 5V, evitar a divisão por zero
            return analogValue, None  # Retornar None para lux para indicar que não conseguimos calcular
        
        denominator = (1 - voltage / 5)
        if denominator == 0:  # Evitar divisão por zero
            return analogValue, None
        
        resistance = 2000 * voltage / denominator
        
        if resistance == 0:  # Evitar divisão por zero na próxima etapa
            return analogValue, None
        
        lux = round(math.pow(RL10 * 1e3 * math.pow(10, GAMMA) / resistance, (1 / GAMMA)), 3)

        return analogValue, lux
    else:
        return None, None

# Configuração inicial
serial_port.write(('pinMode(' + str(led) + ', OUTPUT)').encode())
serial_port.write(('pinMode(' + ldr + ', INPUT)').encode())

while True:
    valorldr, lux_value = read_ldr_and_lux()
    if lux_value is not None:
        # Obtenção da turbidez estimada usando a função de regressão
        turbidez_estimada = regressao_linear(lux_value)
        if turbidez_estimada > 1000:
            continue
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        query = "INSERT INTO turbidimetro (sensoriamento_ldr, turbidez, date_time) VALUES (%s, %s, %s)"
        
        # Altere "lux_value" por "turbidez_estimada" na tupla values
        values = (valorldr, turbidez_estimada, current_time)
        cursor.execute(query, values)
        cnx.commit()
        
        print(f"{current_time} - Valor LDR: {valorldr} | Turbidez Estimada: {turbidez_estimada}")

        serial_port.write(('digitalWrite(' + str(led) + ', HIGH)').encode())

cursor.close()
cnx.close()
